In [2]:
import tensorflow as tf
from tensorflow.keras import layers, Sequential
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os

2023-04-17 12:03:55.299951: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-17 12:03:55.541807: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-04-17 12:03:56.334464: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/dell/miniconda3/envs/tf/lib/:/home/dell/miniconda3/envs/tf/lib/python3.8/site-packages/tensorrt/
2023-04-17 12:03:56.334528: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load d

In [3]:
GPU = tf.config.list_physical_devices()
print(GPU)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-04-17 12:03:57.905836: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-17 12:03:57.924148: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-17 12:03:57.924311: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [4]:
path = '/home/dell/ML/Lung/COVID-19_Radiography_Dataset/'
type_disease = {'COVID': 3616, 'Lung_Opacity':6012,'Normal':10192,'Viral Pneumonia':1345}
mi = ['/masks', '/images']

In [5]:
type_disease['COVID']=3616
type_disease['Lung_Opacity']=3616
type_disease['Normal']=3616

In [6]:
size = 0
for i, j in type_disease.items():
    lung = path+i+mi[1]+'/'+i+'-'
    mask = path+i+mi[0]+'/'+i+'-'
    for c in range(j):
        img_lung = cv2.imread(lung+f'{c+1}.png')
        img_mask = cv2.imread(mask+f'{c+1}.png')
        size_lung = img_lung.shape
        size_mask = img_mask.shape
        bro = max(size_lung[0],size_mask[0])
        if(bro>size):
            size = bro

In [8]:
# img_lung = cv2.imread('/home/dell/ML/Lung/COVID-19_Radiography_Dataset/COVID/images/COVID-1.png')
# img_mask = cv2.imread('/home/dell/ML/Lung/COVID-19_Radiography_Dataset/COVID/masks/COVID-1.png')
# size_lung = img_lung.shape[0]
# size_mask = img_mask.shape[0]
# plt.imshow(img_lung)

In [9]:
# plt.imshow(img_mask)

In [10]:
# l = 299-size_lung
# b = 299-size_mask
# print(l)
# print(b)
# img_lung = cv2.copyMakeBorder(img_lung, l, 0, l, 0,cv2.BORDER_CONSTANT)
# img_mask = cv2.copyMakeBorder(img_mask, b, 0, b, 0, cv2.BORDER_CONSTANT)
# plt.imshow(img_lung)

In [11]:
# plt.imshow(img_mask)

In [12]:
for i, j in type_disease.items():
    lung = path+i+mi[1]+'/'+i+'-'
    mask = path+i+mi[0]+'/'+i+'-'
    save = directory+i 
    os.chdir(save)
    for c in range(j):
        img_lung = cv2.imread(lung+f'{c+1}.png')
        img_mask = cv2.imread(mask+f'{c+1}.png')
        size_lung = img_lung.shape[0]
        size_mask = img_mask.shape[0]
        l = size-size_lung
        b = size-size_mask
        img_lung = cv2.copyMakeBorder(img_lung, 0, l, 0, l,cv2.BORDER_CONSTANT)
        img_mask = cv2.copyMakeBorder(img_mask, 0, b, 0, b, cv2.BORDER_CONSTANT)
        new_img = cv2.bitwise_and(img_lung, img_mask)
        cv2.imwrite(f'{i}-{c+1}.png', new_img)

In [13]:
class_names = list(type_disease.keys())
print(class_names)

['COVID', 'Lung_Opacity', 'Normal', 'Viral Pneumonia']


In [14]:
directory = '/home/dell/ML/Lung/directory/'

In [15]:
x_train = np.zeros((21165,256,256,3), dtype = 'uint8')
y_train = np.zeros((21165,256,256), dtype='bool')

In [16]:
x = '/home/dell/ML/Lung/kill_me/x/'
y = '/home/dell/ML/Lung/kill_me/y/'
n = 0 
for type, count in type_disease.items():
    for i in range(count):
        img_x = cv2.imread(f'{x}{type}-{i+1}.png')
        img_y = cv2.imread(f'{y}{type}-{i+1}.png')
        img_y = cv2.cvtColor(img_y, cv2.COLOR_RGB2GRAY)
        img_x = cv2.resize(img_x, (256, 256))
        img_y = cv2.resize(img_y, (256, 256))
        x_train[n] = img_x
        y_train[n] = img_y
        n+=1

In [17]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model

def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding = 'same')(input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    y = Conv2D(num_filters, 3, padding = 'same')(x)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    return y

def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2,2))(x)
    return x,p

def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2,2), strides = 2, padding = 'same')(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unit(input_shape):
    inputs = Input(input_shape)

    s1,p1 = encoder_block(inputs, 64)
    s2,p2 = encoder_block(p1, 128)
    s3,p3 = encoder_block(p2, 256)
    s4,p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    output = Conv2D(1, 1, padding = 'same', activation = 'sigmoid')(d4)

    model = Model(inputs, output, name = 'u-net')
    return model

In [18]:
model = build_unit((size, size, 3))
model.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)
model.summary()

2023-04-17 12:04:17.498053: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-17 12:04:17.498894: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-17 12:04:17.499060: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-17 12:04:17.499170: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

Model: "u-net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 256, 256, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                             

In [24]:
with tf.device('CPU'):
    model.fit(
        x_train, y_train,
        validation_split = 0.1,
        epochs = 32,
        # callbacks = callbacks,
        batch_size = 2
    )

Epoch 1/32


2023-04-17 12:15:40.142850: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3744989184 exceeds 10% of free system memory.
2023-04-17 12:15:42.010078: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1248329728 exceeds 10% of free system memory.


2327/9524 [======>.......................] - ETA: 5:32:12 - loss: 0.1094 - accuracy: 0.9598

KeyboardInterrupt: 

In [ ]:
model.save('uncomplete unet model.h5')